In [37]:
include("functions.jl")
using Pkg, CPLEX
Pkg.status()

      Status `C:\Users\bhavrathod\Documents\technical_exercise\Project.toml`
  [a076750e] CPLEX v0.9.3
  [336ed68f] CSV v0.10.4
  [9961bab8] Cbc v1.0.1
  [a93c6f00] DataFrames v1.3.4
  [87dc4568] HiGHS v1.1.3
  [4076af6c] JuMP v1.0.0
  [f0f68f2c] PlotlyJS v0.18.8


In [38]:
fp = joinpath("inputs.csv")
data = read_input(fp);
first(data, 10)

┌ Info: # of rows in raw data = 
│   (size(data))[1] = 8760
└ @ Main C:\Users\bhavrathod\Documents\technical_exercise\functions.jl:6
┌ Info: # of rows in daterange = 
│   (size(dr))[1] = 8760
└ @ Main C:\Users\bhavrathod\Documents\technical_exercise\functions.jl:10


,Timestamp,solar_cf_perkw,da_prices,rt_prices
,DateTime,Float64,Float64,Float64
1,2019-01-01T00:00:00,0.0,25.57,7.14
2,2019-01-01T01:00:00,0.0,22.2,13.11
3,2019-01-01T02:00:00,0.0,18.5,22.2
4,2019-01-01T03:00:00,0.0,18.32,27.95
5,2019-01-01T04:00:00,0.0,18.56,3.48
6,2019-01-01T05:00:00,0.0,18.7,17.9
7,2019-01-01T06:00:00,0.0,21.14,13.23
8,2019-01-01T07:00:00,0.0465,21.2,12.92
9,2019-01-01T08:00:00,0.2325,23.19,2.26


## Calculate backcast revenues for proposed solar system

**Solar system configuration**
- Size = 300 MW
- Inverter capacity = 250 MW
- Capex = \$1.35/W-DC

Solar per kW CF is provided in the raw data

Solar O&M and opex costs = 0.0

1. We want to  minimize cost of meeting marginal energy requirement for each hour
2. Assume two generation sources : grid and solar
    1. Both plants can have different generation at each timestep
3. For each timestep, supply >= demand
4. For each timestep, solar_supply <= solar_capacity * cf
5. grid and solar >= 0.0 for each timestep

In [39]:
# How many timesteps do we want to analyze?
upper_lim = 8760

8760

In [40]:
m_solar, x = solar_optimize();

[0.0 0.0 0.0 0.0 0.0; 25.57 22.2 18.5 18.32 18.56]
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Parallel mode: deterministic, using up to 4 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 3 threads...
Tried aggregator 1 time.
LP Presolve eliminated 17520 rows and 17520 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (6.04 ticks)

Dual simplex solved model.



In [41]:
sum(value.(x[:solar, :]))

1839.4190999999903

In [42]:
sum(value.(x[:grid, :]))

6920.580899999963

In [43]:
sum(value.(x[:grid, :])) + sum(value.(x[:solar, :]))

8759.999999999953

In [44]:
objective_value(m_solar)

198335.72169300012

In [45]:
sum(data.da_prices) - objective_value(m_solar)

50850.42830699991

## Calculate backcast revenues for proposed storage system

**Solar system configuration**
- Size = 50 MW
- Capacity = 200 MW
- Efficiency = 100%
- Capex = \$375/kWh

Storage operating costs = 0.0

1. We want to  minimize cost of meeting marginal energy requirement for each hour
2. Assume two generation sources : grid and storage discharge
    1. Both plants can have different generation at each timestep
3. For each timestep, supply >= demand
4. For each timestep, storage_discharge <= storage_capacity * inverter_factor
5. grid and storage discharge >= 0.0 for each timestep

In [46]:
m_storage, a, b, c = storage_optimize();

[0.0 0.0 0.0 0.0 0.0; 25.57 22.2 18.5 18.32 18.56; 25.57 22.2 18.5 18.32 18.56]
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Found incumbent of value 498372.300000 after 0.02 sec. (5.04 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 8761 rows and 8760 columns.
Aggregator did 8760 substitutions.
Reduced MIP has 35045 rows, 35040 columns, and 122640 nonzeros.
Reduced MIP has 17520 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.19 sec. (88.75 ticks)
Probing time = 0.03 sec. (3.70 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 35045 rows, 35040 columns, and 122640 nonzeros.
Reduced MIP has 17520 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.34 sec. (597.78 ticks)
Probing time = 0.05 sec. (4.41 ticks)
Clique table members: 8760.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.89 sec. (45

In [47]:
sum(value.(a[:stor_disch, :]))

4379.755

In [48]:
sum(value.(a[:stor_ch, :]))

4379.745

In [49]:
objective_value(m_storage)

176680.22239999997

In [50]:
sum(value.(b[:bess_mwh, :]))

4379.755

In [52]:
m_both, k, l, m = solar_storage();
objective_value(m_both)

[0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 25.57 22.2 18.5 18.32 18.56; 25.57 22.2 18.5 18.32 18.56]
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Found incumbent of value 447521.871693 after 0.02 sec. (6.01 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 13098 rows and 13097 columns.
Aggregator did 8760 substitutions.
Reduced MIP has 39468 rows, 39463 columns, and 131486 nonzeros.
Reduced MIP has 17520 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.27 sec. (99.76 ticks)
Probing time = 0.03 sec. (4.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4929 rows and 4423 columns.
MIP Presolve modified 7834 coefficients.
Reduced MIP has 34539 rows, 35040 columns, and 121628 nonzeros.
Reduced MIP has 17520 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (54.53 ticks)
Probing time = 0.03 sec. (3.66 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 34539 rows, 35040 columns, and 121628 nonzeros.
Reduced

137909.32744299993

In [53]:
sum(value.(k[:stor_disch, :]))

3517.2441999999955

In [55]:
sum(value.(k[:stor_ch, :]))

3517.234199999998

In [56]:
sum(data.da_prices)

249186.15000000002